In [ ]:
# All Rights Reserved.
#
# Written by Vinicius Ribeiro <sts.viniciusr@gmail.com>,
#            Marcos Santos <marcosdossantos_doutorado_uff@yahoo.com.br>,
#            Oct 2022.
#

# Python.
import IPython
import functools
import typing as t

# Libs.
import ipysheet
import ipywidgets
import numpy as np

# Global database.
SHEET = None

# Tamanho das bases de dados apresentadas na saída.
_WIDTH = '1000px'

#  Estilização utilizada para cabeçalhos.
_HDR_STYLE = {'textAlign': 'center', 'fontWeight': 'bold'}

# Configuração padrão para novas células.
_CELL_DEFAULT_ARGS = {
    'read_only': True, 'background_color': '', 'style': {'textAlign': 'center'}
}

# Alias para novas células ipysheet.
_CELL = functools.partial(ipysheet.cell, **_CELL_DEFAULT_ARGS)

# Alias para conversão de numpy arrays.
_ARR = functools.partial(np.array, dtype=np.float64)

# Mensagem de erro estilizada.
_MSG = '<p><span style="color:#FF0000"><strong>{}</strong></span></p>'

# Alias para interpretador de texto em HTML.
_HTML = functools.partial(IPython.display.HTML)

_INPUT = ipywidgets.Output()
_OUTPUT = ipywidgets.Output()

@_OUTPUT.capture(clear_output=True)
@_INPUT.capture(clear_output=True)
def limpar_saidas(*kwa, **args) -> None:
    '''Limpa toda a area de trabalho.'''

    global SHEET

    SHEET = None

@_OUTPUT.capture(clear_output=True)
@_INPUT.capture(clear_output=True)
def gera_base(btn, form) -> ipysheet.sheet:
    '''Reconstrói a planilha-base de entrada de dados para o algoritmo.'''

    global SHEET

    # Feedback no botão.
    btn.icon = 'fa-spinner'
    btn.disabled = True

    st = {}

    # Os cabeçalhos são desativados, e é adicionada uma linha e uma coluna para
    # que possam ser nomeadas as alternativas e critérios, respectivamente.
    #
    st['column_headers'] = False
    st['row_headers'] = False

    # Duas linhas devem ser adicionadas para seleção do tipo de otimização e
    # nomes dos critérios.
    #
    st['rows'] = form['alternativas'].value + 2

    # Uma coluna é adicionada para nomes das alternativas.
    #
    st['columns'] = form['criterios'].value + 1

    SHEET = ipysheet.sheet(**st)

    SHEET.column_width = [10] + ([5] * form['criterios'].value)
    SHEET.layout = ipywidgets.Layout(width=_WIDTH, height='100%')

    # 2.1. Adiciona a primeira coluna:
    #
    #   - Tipo da Otimização;
    #   - Alternativa / Critério;
    #   - Nome da alternativa 0.
    #   ...
    #   - Nome da alternativa N.
    #
    _CELL(0, 0, 'TIPO DA OTIMIZAÇÃO', style=_HDR_STYLE)
    _CELL(1, 0, 'Alternativa / Critério')

    for row in range(form['alternativas'].value):
        ipysheet.row(
            row + 2,
            [f'ALTERNATIVA {row+1}'] + [''] * form['criterios'].value,
            read_only=False, style={'textAlign': 'center'}
        )

    # Processa as colunas, uma a uma.
    #
    for c in range(form['criterios'].value):
        # Uma coluna de títulos foi definia em 2.1, por poranto as referências
        # começam em c+1.
        #
        _CELL(0, c + 1, 'MAX', choice=['MAX', 'MIN'], style=_HDR_STYLE, read_only=False)
        _CELL(1, c + 1, f'CRITÉRIO {c + 1}', background_color='#DEDAD9', read_only=False)

    with _INPUT:
        IPython.display.display(_HTML('<h1 style="text-align: center;"><strong>AHP Gaussiano</strong></h1>'))

        IPython.display.display(_HTML('<h4>Nomeie as alternativas, os crit&eacute;rios, preencha a matriz e clique no bot&atilde;o <span style="background-color: #eeeeee; color: #737373; display: inline-block; padding: 3px 10px; border-radius: 5px;">Clacular</span>.</p><p>&nbsp;</h4>'))

        IPython.display.display(SHEET)

        # Feedback no botão.
        btn.disabled = False
        btn.icon = 'check'

@_OUTPUT.capture(clear_output=True)
def processa(btn) -> None:
    '''Aplica o algoritmo.'''

    global SHEET

    def _valida_coluna(col) -> bool:
        '''Valida dados de entrada de critérios (colunas da base).'''

        bit = True

        def _represents_int(s) -> bool:
            try:
                s = np.float64(s)

                return True

            except ValueError:

                return False

        otimizacao = col[0]

        if otimizacao not in ['MAX', 'MIN']:
            if col[0] != 'TIPO DA OTIMIZAÇÃO':
                IPython.display.display(_HTML(_MSG.format(f'Tipo de otimização invalido para critério {col[1]}')))

                bit = False

        elif any(x is None for x in col[2:]):
            IPython.display.display(_HTML(_MSG.format(f'Sem valor em célula de "{col[1]}".')))

            bit = False

        elif any(not x and otimizacao == 'MIN' for x in col[2:]):
            IPython.display.display(_HTML(_MSG.format(f'Valor "zero" alocado na minimização do critério "{col[1]}".')))

            bit = False

        elif [x for x in col[2:] if not _represents_int(x)]:
            ns = [x for x in col[2:]][0]
            IPython.display.display(_HTML(_MSG.format(f'O valor "{ns}" não é inteiro. "{type(ns)}"')))

            bit = False

        return bit

    def mostra_resultados() -> None:
        '''Monta a apresentação de todos os resultados.'''

        # Feedback no botão.
        btn.icon = 'fa-spinner'
        btn.disabled = True

        # 1. Matriz normalizada.
        #
        parametros: t.Dict[str, t.Any] = {}

        parametros['row_resizing'] = True
        parametros['row_headers'] = False
        parametros['columns'] = len(matriz_decisao)
        parametros['rows'] = len(matriz_decisao[0]) - 2
        parametros['column_headers'] = [''] + list(matriz_decisao[1:, 1])

        tabela1 = ipysheet.sheet(**parametros)

        # 1.1. Preenche coluna-cabeçalho da matriz normalizada.
        ipysheet.column(
            0, matriz_decisao[0][2:], read_only=True,
            style={'textAlign': 'center'}, background_color='#DEDAD9'
        )

        # 1.2 Atribui as células númericas.
        ipysheet.cell_range(
            matriz_normalizada[1:, 2:].transpose(), column_start=1,
            type='numeric', **_CELL_DEFAULT_ARGS
        )

        # 2. Matriz de medidas para critérios.
        #
        parametros['rows'] = 4
        parametros['columns'] = len(matriz_decisao)
        parametros['column_headers'] = [''] + list(matriz_decisao[1:, 1])

        tabela2 = ipysheet.sheet(**parametros)

        # 2.1. Preenche a matriz.
        ipysheet.cell_range(
            [
                (['Média'] + list(media)),
                (['Desvio Padrão'] + list(desvio_padrao)),
                (['Coeficiente de Variação'] + list(fator_gaussiano)),
                (['Coeficiente de Variação Normalizado'] + list(fator_gaussiano_normalizado))
            ], type='numeric', style={'textAlign': 'center'}
        )

        # 3. Matriz de resultado.
        #
        parametros['columns'] = 3
        parametros['rows'] = len(matriz_normalizada[0, 2:])
        parametros['column_headers'] = ['Alternativa', 'AHP-G', 'RANK']

        tabela3 = ipysheet.sheet(**parametros)

        for idx, alternativa in enumerate(matriz_normalizada[0, 2:]):
            # Pinta a melhor opção de verde, e a pior de vermelho.
            #
            bg = ''

            if ranking[idx] == 1:
                bg = 'green'

            elif ranking[idx] == len(ranking):
                if len(ranking) > 1:
                    bg = 'red'

            ipysheet.cell(
                idx, 0, alternativa, read_only=True,
                style={'textAlign': 'center'}, background_color=bg
            )
            ipysheet.cell(
                idx, 1, soma_ponderacoes[idx], background_color=bg,
                read_only=True, type='numeric', style={'textAlign': 'center'}
            )
            ipysheet.cell(
                idx, 2, ranking[idx], numeric_format='0', background_color=bg,
                type='numeric', style={'textAlign': 'center'}, read_only=True
            )

        # Configura a exibição de todos os resultados.
        #
        saidas = ipywidgets.GridspecLayout(2, 2)

        saidas[0, 0] = tabela1
        saidas[0, 0].column_width = [5] + ([3] * form['criterios'].value)

        saidas[0, 1] = tabela3
        saidas[0, 1].column_width = [5, 3, 3]

        saidas[1, 0] = tabela2
        saidas[1, 0].column_width = [5] + ([3] * form['criterios'].value)

        IPython.display.display(saidas)

        # Feedback no botão.
        btn.disabled = False
        btn.icon = 'check'

    if not SHEET:
        return

    ##########
    # Passo 1.
    #
    # Determinação da Matriz de Decisão.
    #
    # 1.1. Acessa dados da base inputada no painel interativo.
    # 1.2. Normalização dos valores.
    #
    # 1.1.1. Coleta dos dados em tela, carregados como uma matriz numpy.
    #
    matriz_decisao = ipysheet.to_array(SHEET).transpose()

    # 1.1.2. Declara uma réplica base, que será normalizazda a seguir.
    #
    matriz_normalizada = matriz_decisao.copy()

    # 1.2. Itera a matriz de decisão, coluna por coluna, normalizando-a e
    # gravando a matriz normalizada.
    #
    for c_idx, col in enumerate(matriz_decisao[1:], 1):
        # 1.2.1. Valida se as entradas do usuário são válidas.
        #
        if not _valida_coluna(col):
            return

        # 1.2.2. Dados recebidos do módulo de interação tem o tipo textual,
        # então os valores numéricos acessados são devidamente convertidos
        # para o tipo correto, viabilizando a computação.
        #
        vals = _ARR(col[2:])

        for r_idx, v in enumerate(vals, 2):
            x = v / sum(vals) if col[0] == 'MAX' else (1 / v) / sum(1 / vals)

            matriz_normalizada[c_idx][r_idx] = x

    ##########
    # Passo 2.
    #
    #  Cálculo do fator gaussiano para cada critério.
    #
    media = _ARR(matriz_normalizada[1:, 2:]).mean(axis=1)
    desvio_padrao = _ARR(matriz_normalizada[1:, 2:]).std(axis=1, ddof=1)

    # 2.1. Calcula o fator gaussiano.
    fator_gaussiano = desvio_padrao / media

    # Normalizando o fator gaussiano.
    fator_gaussiano_normalizado = fator_gaussiano / sum(fator_gaussiano)

    ##########
    # Passo 3.
    #
    # Normalização dos resultados.
    #
    #
    # 3.1. Transpõe a matriz normalizada para calcular orientado à linhas.
    #
    aux = matriz_normalizada[1:, 2:].transpose().astype('float64')

    # 3.2. Calcula a soma das ponderações;
    #
    soma_ponderacoes = (aux * fator_gaussiano_normalizado).sum(axis=1)

    # 3.3. Computa o resultado de ranking (ordem das alternativas);
    #
    indices = np.argsort(-soma_ponderacoes)
    ranking = np.empty_like(indices)
    ranking[indices] = np.arange(len(soma_ponderacoes))+1

    ##########
    # Apresentação.
    #
    with _OUTPUT:
        return mostra_resultados()

# Widgets.
panel = []
form: t.Dict[str, t.Any] = {}
grid = ipywidgets.GridspecLayout(2, 3)

# 1.1. Número de alternativas.
form['alternativas'] = {}
form['alternativas']['min'] = 2
form['alternativas']['max'] = 10
form['alternativas']['value'] = 2
form['alternativas']['disabled'] = False
form['alternativas']['description'] = 'Núm. de Alternativas (Linhas)'
form['alternativas']['style'] = {'description_width': 'initial'}
form['alternativas'] = ipywidgets.BoundedIntText(**form['alternativas'])

# 1.2. Número de Critérios.
form['criterios'] = {}
form['criterios']['min'] = 1
form['criterios']['max'] = 10
form['criterios']['value'] = 2
form['criterios']['disabled'] = False
form['criterios']['description'] = 'Núm. de Critérios (Colunass)'
form['criterios']['style'] = {'description_width': 'initial'}
form['criterios'] = ipywidgets.BoundedIntText(**form['criterios'])

# 2. Create the widgets Grid.
grid[0, 0] = form['alternativas']
grid[1, 0] = form['criterios']

grid[0, 1] = ipywidgets.Button(description='Gerar Base')
grid[0, 1].on_click(functools.partial(gera_base, form=form))

grid[1, 1] = ipywidgets.Button(description='Calcular')
grid[1, 1].on_click(functools.partial(processa))

grid[0, 2] = ipywidgets.Button(description='Limpar Tudo')
grid[0, 2].on_click(limpar_saidas)

# Painel principal. {{{
#
# Onde são exibidos os controles.
#
panel.append(ipywidgets.interactive_output(limpar_saidas, form))
panel.append(grid)
panel.append(_INPUT)
panel.append(_OUTPUT)

IPython.display.display(ipywidgets.VBox(panel))
# }}}
